In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dolfinx as dfx

import h5py

from dolfinx.fem.petsc import NonlinearProblem

%matplotlib widget
from matplotlib import pyplot as plt
# plt.style.use('fivethirtyeight')

from mpi4py import MPI

import numpy as np

from pathlib import Path

import scipy as sp

import ufl

from pyMoBiMP.cahn_hilliard_utils import (
    cahn_hilliard_mu_form,
    cahn_hilliard_dydt_form,
    charge_discharge_stop, 
    AnalyzeOCP,
    y_of_c,
    c_of_y,
    populate_initial_data)

from pyMoBiMP.fenicsx_utils import (evaluation_points_and_cells,
                           get_mesh_spacing,
                           time_stepping,
                           NewtonSolver,
                           FileOutput,
                           Fenicx1DOutput)

from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

from pyMoBiMP.plotting_utils import (
    add_arrow, 
    plot_charging_cycle, 
    plot_time_sequence,
    PyvistaAnimation,
    animate_time_series)

comm_world = MPI.COMM_WORLD

In [ ]:
# Discretization
# --------------

# Set up the mesh
n_elem = 128

mesh = dfx.mesh.create_unit_interval(comm_world, n_elem)

dx_cell = get_mesh_spacing(mesh)

print(f"Cell spacing: h = {dx_cell}")

# For later plotting use
x = np.linspace(0, 1, 101)
points_on_proc, cells = evaluation_points_and_cells(mesh, x)

# Initial timestep size
dt = dfx.fem.Constant(mesh, dx_cell * 0.01)

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

mixed_element = elem1

V = dfx.fem.FunctionSpace(mesh, mixed_element)  # A single-component function space

In [ ]:
# The mixed-element functions
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

In [ ]:
# Compute the chemical potential df/dc
a = 6. / 4
b = 0.2
cc = 5

# a = 5. # 6. / 4
# b = 0. # 0.2
# cc = 0 # 5

free_energy = lambda u, log, sin: u * log(u) + (1-u) * log(1-u) + a * u * (1 - u) + b * sin(cc * np.pi * u)

eps = 1e-4

res_c_left = sp.optimize.minimize_scalar(
    lambda c: free_energy(c, np.log, np.sin),
    bracket=(2 * eps, 0.05),
    bounds=(eps, 0.05))

assert res_c_left.success

c_left = res_c_left.x

res_c_right = sp.optimize.minimize_scalar(
    lambda c: free_energy(c, np.log, np.sin),
    bracket=(0.95, 1 - 2 * eps),
    bounds=(0.95, 1 - eps))

assert res_c_right.success

c_right = res_c_right.x

fig, ax = plt.subplots()

eps = 1e-3

c_plot = np.linspace(eps, 1-eps, 200)

ax.plot(c_plot, free_energy(c_plot, np.log, np.sin))

ax.scatter(c_left, free_energy(c_left, np.log, np.sin))
ax.scatter(c_right, free_energy(c_right, np.log, np.sin))

ax.set_xlabel(r"$c$")
ax.set_ylabel(r"$F(c)$")

plt.show()

In [ ]:
# Experimental setup
# ------------------

# charging current
I_charge = dfx.fem.Constant(mesh, 1e-1)

T_final = 2. / I_charge.value  # ending time

def experiment(t, u, I_charge, **kwargs):

    return charge_discharge_stop(t, u, I_charge, c_of_y = c_of_y, **kwargs)

event_params = dict(I_charge=I_charge, stop_on_full=False, stop_at_empty=False, cycling=False, logging=True)

In [ ]:
# The variational form
# --------------------

params = dict(I_charge=I_charge, grad_c_bc=lambda c: 0.1 * I_charge)

y = dfx.fem.Function(V)
mu = dfx.fem.Function(V)

residual_mu = cahn_hilliard_mu_form(
    y,
    c_of_y=c_of_y,
    free_energy=lambda c: free_energy(c, ufl.ln, ufl.sin))

# mu == f_A - gamma * Delta c
problem_mu = dfx.fem.petsc.LinearProblem(
    ufl.lhs(residual_mu), ufl.rhs(residual_mu))

residual_dydt = cahn_hilliard_dydt_form(
    y, mu, I_charge, c_of_y=c_of_y
)

# dcdt = dcdy * dydt == div (M grad mu)
problem_dydt = dfx.fem.petsc.LinearProblem(
    ufl.lhs(residual_dydt),
    ufl.rhs(residual_dydt))

In [ ]:
# Initial data
# ------------

y_ini = dfx.fem.Function(V)

# Constant
c_ini_fun = lambda x: 1e-3 * (2. - np.cos(x[0] * np.pi))
# c_ini_fun = lambda x: 1e-3 * np.ones_like(x[0])

c_ini = dfx.fem.Function(V)
c_ini.interpolate(lambda x: c_ini_fun(x))

y_ini.interpolate(dfx.fem.Expression(y_of_c(c_ini), V.element.interpolation_points()))

I_charge.value = 0.  # for this testing purpose

y.interpolate(y_ini)
mu_ini = problem_mu.solve()

mu.interpolate(mu_ini)
dydt_ini = problem_dydt.solve()

## Figure 1: initial data
plt.figure()

plt.plot(x, y_ini.eval(points_on_proc, cells), label="y")
plt.plot(x, mu_ini.eval(points_on_proc, cells), label=r"$\mu$")
plt.plot(x, c_ini.eval(points_on_proc, cells), label="c")

plt.legend()

plt.show()

## Figure 2: evaluation of time derivative
plt.figure()

plt.plot(x, dydt_ini.eval(points_on_proc, cells), label="dydt")

plt.legend()

plt.show()

In [ ]:
# From: https://towardsdatascience.com/do-stuff-at-each-ode-integration-step-monkey-patching-solve-ivp-359b39d5f2

from scipy.integrate._ivp.base import OdeSolver  # this is the class we will monkey patch

from tqdm import tqdm

### monkey patching the ode solvers with a progress bar

# save the old methods - we still need them
old_init = OdeSolver.__init__
old_step = OdeSolver.step

# define our own methods
def new_init(self, fun, t0, y0, t_bound, vectorized, support_complex=False):

    # define the progress bar
    self.pbar = tqdm(total=t_bound - t0, unit='ut', initial=t0, ascii=True, desc='IVP')
    self.last_t = t0

    # call the old method - we still want to do the old things too!
    old_init(self, fun, t0, y0, t_bound, vectorized, support_complex)


def new_step(self):
    # call the old method
    old_step(self)

    # update the bar
    tst = self.t - self.last_t
    self.pbar.update(tst)
    self.last_t = self.t

    # close the bar if the end is reached
    if self.t >= self.t_bound:
        self.pbar.close()


# overwrite the old methods with our customized ones
OdeSolver.__init__ = new_init
OdeSolver.step = new_step

def rhs_func(t, y_vec):
    y.x.array[:] = y_vec
    mu_ = problem_mu.solve()

    mu.x.array[:] = mu_.x.array[:]
    dydt_ = problem_dydt.solve()

    # Fix to stabilize r=0 behavior. By copying the inner-next value
    # we enforce first-order Neuman conditions to the time derivative
    dydt_.x.array[0] = dydt_.x.array[1]

    return dydt_.x.array[:]

In [ ]:
I_charge.value = 0.01
T_final = 20.

solution = sp.integrate.solve_ivp(
    rhs_func, [0, T_final], y_ini.x.array[:],
    first_step=1e-9,
    max_step=1e-3,
    min_step=1e-12,
    t_eval = np.linspace(0, T_final, 20), 
    method='LSODA',
    dense)

solution.message

### Some questions to answer:

- Is the timestepping method adaptive to phase changes
- is it worth using `DenseOutput`?

In [ ]:
fig = plt.figure()

plt.plot(solution.y)

plt.show()

In [ ]:
import pyvista as pv

cell, types, x = dfx.plot.vtk_mesh(mesh)

chart = pv.Chart2D()

def y_of_y(y): return np.exp(y) / (1. + np.exp(y))

for it, t in enumerate(solution.t):

    y = solution.y[:, it]
    c = y_of_y(y)

    chart.line(x[:, 0], c, color = (t / solution.t[-1], 0, 0))

plotter = pv.Plotter()
plotter.add_chart(chart)

plotter.show()